In [16]:
#import sys
import requests
import time
import datetime
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as md
from pandas.io.json import json_normalize
import pandas as pd

In [30]:
endpoint_url = "https://api.whale-alert.io/v1/transactions?"
# Our testing api-key
headers = {"X-WA-API-KEY" : "WZmlatDSHiZJjbb7JeLSub1fGjb9qwnd"}
# define the parameters of the request
st_time = int(time.time() - 3590)
params = {
    "min_value" : 500000,
    "start" : st_time 
    }
#url = endpoint_url + params["commodity"] + "." + params["data_version"] + "/exchanges/" + params["exchange"] + "/" + params["instrument_class"] + "/" + params["instrument"] + "/trades/recent"
# for full historical trade function, have "trades{?start_time,end_time,page_size,continuation_token}" after "instrument"
# for recent trades, have "recent{?limit}" after "trades"
# make the request
rcomp = requests.get(endpoint_url, headers = headers, params = params)
raw_query = rcomp.json()
#response = raw_query["data"]
#crypto_data = pd.DataFrame(response)
#crypto_data.to_csv('/Users/apple/Desktop/kaiko_test/test.csv)

In [31]:
raw_query

{'result': 'success',
 'cursor': '18dfabf7-18dfabf7-5dcc0f68',
 'count': 25,
 'transactions': [{'blockchain': 'ethereum',
   'symbol': 'usdt',
   'id': '417280047',
   'transaction_type': 'transfer',
   'hash': 'df20278bf267897719f9618699fe5753b85a83d843547db371b098f2d06bd2a8',
   'from': {'address': 'a97db957ee3e0d7bcf74727e2bfda11c628157a8',
    'owner_type': 'unknown'},
   'to': {'address': '079187c39e2c67a3395a98e39d7ea582efb3cf73',
    'owner_type': 'unknown'},
   'timestamp': 1573650836,
   'amount': 2372763.8,
   'amount_usd': 2364617.5,
   'transaction_count': 1},
  {'blockchain': 'ethereum',
   'symbol': 'usdt',
   'id': '417280730',
   'transaction_type': 'transfer',
   'hash': '68bc180f58ea2d7083d4d8a8aa9845ba069f0f0e7d9404f8d074ba6a53380229',
   'from': {'address': '53b93c55a2357ac2b94e3a3bd5c4595e1fd2fa50',
    'owner_type': 'unknown'},
   'to': {'address': '958eba60553c4ada78a85dad13bff54df7a54748',
    'owner': 'bitfinex',
    'owner_type': 'exchange'},
   'timestamp': 1